# Исследование надежности заемщиков


## Изучение данных таблицы

Импортирт библиотеки pandas

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Вывод первых 20 строчек датафрейма `data` 

In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Основную информацию о датафрейме 

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Количество пропущенных значений для каждого столбца

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

 В двух столбцах есть пропущенные значения. Один из них — `days_employed`.Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаются артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально.

In [ ]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведеим медианное значение трудового стажа `days_employed` в днях

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

Перечень уникальных значений столбца `children`.

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Пропуски в столбце `days_employed` заполняем медианными значениями по каждого типа занятости `income_type`.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Выведем колличсво дупликатов и удалим строки где они есть

In [ ]:
data.duplicated().sum()

54

In [ ]:
data = data.drop_duplicates()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,categorize_children
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,1 ребенок
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,1 ребенок
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,нет детей
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,многодетная семья
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,нет детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью,1 ребенок
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем,нет детей
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью,1 ребенок
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем,многодетная семья


Обработка неявных дубликатов в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. 

In [ ]:
data['education'] = data['education'].str.lower()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Перечень уникальных целей взятия кредита из столбца `purpose`

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

 Функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуйте данные и ответьте на вопросы

1. children — количество детей в семье
1. days_employed  — общий трудовой стаж в днях
1. dob_years' — возраст клиента в годах
1. education — уровень образования клиента
1. education_id — идентификатор уровня образования
1. family_status — семейное положение
1. family_status_id — идентификатор семейного положения
1. gender — пол клиента
1. income_type — тип занятости
1. debt — имел ли задолженность по возврату кредитов
1. total_income — ежемесячный доход
1. purpose — цель получения кредита
1. total_income_category - категория ежемесячного дохода
1. purpose_category - категория целей получения кредита


Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
import pandas as pd #постоянно то убирала то добавляла библиотеку seaborn и в результате запуталась. Спасибо!

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21348 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21348 non-null  int64  
 1   days_employed          21348 non-null  float64
 2   dob_years              21348 non-null  int64  
 3   education              21348 non-null  object 
 4   education_id           21348 non-null  int64  
 5   family_status          21348 non-null  object 
 6   family_status_id       21348 non-null  int64  
 7   gender                 21348 non-null  object 
 8   income_type            21348 non-null  object 
 9   debt                   21348 non-null  int64  
 10  total_income           21348 non-null  int64  
 11  purpose                21348 non-null  object 
 12  total_income_category  21348 non-null  object 
 13  purpose_category       21348 non-null  object 
dtypes: float64(1), int64(6), object(7)
memory usage: 2.4+ 

In [ ]:
def categorize_children(amount):
    try:
        if amount == 0:
            return 'нет детей'
        elif amount == 1:
            return '1 ребенок'
        elif amount == 2:
            return '2 ребенка'
        elif amount == 3 or amount == 4 or amount == 5:
            return 'многодетная семья'
    except:
        return 'нет категории'

data['categorize_children'] = data['children'].apply(categorize_children)

In [ ]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,categorize_children
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,1 ребенок
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,1 ребенок
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,нет детей
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,многодетная семья
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,нет детей


In [ ]:
y= data.groupby('categorize_children').agg({'debt':['sum', 'count']})
y['sum_count'] = y['debt']['sum'] / y['debt']['count'] * 100
y['total_debt'] = y['debt']['sum'] / len(data) * 100
y.sort_values(by='sum_count', ascending=False).reset_index()

categorize_children  debt        sum_count total_debt
                        sum  count                     
0           2 ребенка   194   2052  9.454191   0.908750
1           1 ребенок   444   4809  9.232689   2.079820
2   многодетная семья    31    380  8.157895   0.145213
3           нет детей  1063  14107  7.535266   4.979389

In [ ]:
#отношение должников по кредитам ко всеv
x = data.groupby('children').agg({'debt':['sum', 'count']})
x['sum_count'] = x['debt']['sum'] / x['debt']['count'] * 100
x['total_debt'] = x['debt']['sum'] / len(data) * 100
x.sort_values(by='sum_count', ascending=False).reset_index()

children  debt        sum_count total_debt
             sum  count                     
0        4     4     41  9.756098   0.018737
1        2   194   2052  9.454191   0.908750
2        1   444   4809  9.232689   2.079820
3        3    27    330  8.181818   0.126476
4        0  1063  14107  7.535266   4.979389
5        5     0      9  0.000000   0.000000

**Вывод:** 
* Наличие задолжностей по крединам зависит от коллчиесва детей в семье.
* Доля задолжностей по кредитам в семьях сдетьми больше чем в семьях где детей нет.
* Самые большие доли занимают семьи с 1-м или 2-мя детьми в семье.

Кредитополучатели без детей являются более ответственными, чем клиенты, у которых есть дети. Видимо, наличие ребенка накладывает на клиента дополнительные финансовое бремя (обязательства) и клиенты либо менее ответственно относятся к своим кредитным обязательствам, либо не имеют возможности в срок оплачивать обязательства.

**Самые большие доли занимают семьи с 1-м или 2-мя детьми в семье.** - возможно, что для них рождение ребенка — это некий шок, влияющий на исполнение финансовых обязательств. В то время как для семей с большим количеством детей - дети стали обыденностью.

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
family_status_debt = data.groupby('family_status').agg({'debt':['sum', 'count']})
family_status_debt['sum_count'] = family_status_debt['debt']['sum'] / family_status_debt['debt']['count'] * 100
family_status_debt['total_debt'] = family_status_debt['debt']['sum'] / len(data) * 100
family_status_debt.sort_values(by='sum_count', ascending=False)

debt        sum_count total_debt
                       sum  count                     
family_status                                         
Не женат / не замужем  273   2796  9.763948   1.278808
гражданский брак       385   4146  9.286059   1.803448
женат / замужем        927  12266  7.557476   4.342327
в разводе               84   1189  7.064760   0.393479
вдовец / вдова          63    951  6.624606   0.295110

**Вывод:** 
* У семей со статусом Не женат / не замужем самая большая доля по невозвратам кредитов 9.7%, к этой катеорги можно отнести гражданский брак, такие показатели могут быть связаны со статусом заработка семьи или целями взятия кредита.
* Средний показатель у семей со статусом женат / замужем 7.5%
* Самый низкий у семей вдовец / вдова 6.6%


**Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [ ]:
add_status_data = data.pivot_table(index=['total_income_category'], values='debt', aggfunc={'count', 'sum'})

In [ ]:
add_status_data['sum_count'] = add_status_data['sum'] / add_status_data['count'] * 100 
add_status_data['total_debt'] = add_status_data['sum'] / len(data) * 100
add_status_data.sort_values(by='sum_count', ascending=False).reset_index()

,total_income_category,count,sum,sum_count,total_debt
0,E,22,2,9.090909,0.009369
1,C,15938,1353,8.489145,6.337830
2,A,25,2,8.000000,0.009369
3,B,5014,354,7.060231,1.658235
4,D,349,21,6.017192,0.098370


**Вывод:** 
Самая большая доля процентов по невозрату у семей с уровнем дохода Е 9.0% и С 8.4%, но их доля Е в таблице очень.
Среднее значение у семей с высоким доходом категории А 8.0%, но их коллчество относительно остальных категорий так же мало. 
Самая низкая у семей с категорие D 6.0% и B 7.0%.
Исходя из здравого смысла логично предположить, что чаще всего должниками становятся люди с небольшим доходом, которым финансовые проблемы не позволяют вовремя осуществлять платежи. Но глядя на наши результаты лично мой вывод таков: выборка по заемщикам, как с наименьшим доходом, так и с наибольшим крайне недостаточна для формирования однозначных выводов. Можно думать, что E-категория самая низкооплачиваемая не может корректно распоряжаться деньгами и «влезает» в кредиты, но тогда неясно почему у А-категории также достаточно высокий процент задолженности. В общем вывод по группам А и Е неочевиден, мало данных.

Если же взять для анализа только группы В, С и D, то по полученным данным не наблюдаем зависимости "те кто получает больше ЗП, лучше возвращает кредит». В целом это логично, так как материальный достаток - не единственная переменная которая влияет на качество кредитной истории.

**Задание 22. Как разные цели кредита влияют на его возврат в срок?**

In [ ]:

add_purpose_data = data.pivot_table(index=['purpose_category'], values='debt', aggfunc={'count', 'sum'})

In [ ]:
add_purpose_data['status_fam'] = add_purpose_data['sum'] / add_purpose_data['count'] * 100 
add_purpose_data['total_debt'] = add_purpose_data['sum'] / len(data) * 100
add_purpose_data.sort_values(by='status_fam', ascending=False)

,count,sum,status_fam,total_debt
purpose_category,,,,
операции с автомобилем,4281,400,9.343611,1.873712
получение образования,3989,369,9.250439,1.728499
проведение свадьбы,2324,183,7.874355,0.857223
операции с недвижимостью,10754,780,7.253115,3.653738


**Вывод:** Больше всего операций проведено с недвижимостью и у них самая низкая доля по возвратам, такая же ситуация и с проведением свадьбы. Больше всего задожностей имеется у операций с автомобилями и получением образования.	

## Общий вывод.

**Общий вывод:**

Было проведено исследование, для изучения влияния различных факторов на погашение кредита в срок. Входные данные - статистика о платеже способности клиентов. Результаты исследования бдудут учтены при построении модели кредитного скоринга - система оценки заемщика, с помощью которой банки предсказывают, насколько аккуратно человек будет выплачивать кредит.

Перед проведением исследования было поставлено несколько гипотез:

* Гипотеза 1: коллчиесво детей влияет на факт погашения кредита в скрок;
* Гипотеза 2: семейное положение влияет на факт погашения кредита в скрок;
* Гипотеза 3: семейный уровень дохода влияет на факт погашения кредита в скрок;
* Гипотеза 4: цель взятия кредита влияет на факт погашения кредита в скрок.

В результате исследования удалось выявить зависимость.

**Гипотеза 1 частично подтверждена:** *для более точных выводов необходимо больше данныхъ по семьям с 3, 4 и 5 детьми.

* 7.5% - доля просроченных кредитов у заевмщиков с без детей;
* 9.5% - доля просроченных кредитов у заевмщиков с 2 детьми;
* 9.2% - доля просроченных кредитов у заевмщиков с 1 детьми.

Рекомендация: провести доополнительное исследование на большей выборке.

**Гипотеза 2 подтверждена:** 
* У клиентов со статусом Не женат / не замужем (9.7%), к ним можно отнести гражданский брак (9.3%), у этих  групп самая большая доля задолжностей по крединам;
* Категорий которые находяться в разводе(7.1%), в статусе женат/замужем (7.5%) или вдовец/вдова (6.6%), вероятность стать долджниками меньше всего.

Рекомендация: разница между 2 пунктами составляет меньше 3%%, я бы не рекомендовала наделять эту зависимость высоким весом при принятии решения о кредито способности.
 
**Гипотеза 3 частично подтверждена:** *для более точных выводов необходимо больше данныхъ по семьям по клиентам с уровнем дохода Е (до 30000), D (от 30001 до 50000) и А (от 1000001)

* 8.5% - доля заемщиков по кредитам с уровнем дохода от 50001 до 200000;
* 7.1% - доля заемщиков по кредитам с уровнем дохода от 200001 до 1000000.

Рекомендация: провести доополнительное исследование на большей выборке.

**Гипотеза 4 подтверждена:**

* 9.3% - доля заемщиков по кредитам, целью которых были операции с автомобилем;
* 9.3% - доля заемщиков по кредитам, целью которых были операции с получением образования;
* 7.8% - доля заемщиков по кредитам, целью которых были операции с проведением свадьбы;
* 7.2% - доля заемщиков по кредитам, целью которых были операции с недвижимостью.

Рекомендация: Разница между 1 и 4 пунктами всего 2%% ,я бы не рекомендовала наделять эту зависимость высоким весом при принятии решения о кредито способности.

**Общие рекомендации:**

1. Специалистам, которые занимаются построением модели кредитного скорингаБ можно использовать зависимость из второй и четвертой гипотез (семейное положение и цели кредита)
1. Необходимо собрать более полную выборку по первой и третьей гепотизам для повторной проверки.

In [ ]:
#all_table = data.pivot_table(index = ['family_status', 'purpose_category'], columns='total_income_category', values='debt', aggfunc={'count', 'sum'})
#all_table